In [1]:
# Installing the required libraries
import os
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.features import rasterize
from shapely.geometry import box
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Steps: 
1. **Load Census Tracts and Taxi Zones:**
    * Load the shapefiles for census tracts and taxi zones.
    * Ensure the merged GeoDataFrame contains population density information for each census tract.

2. **Rasterize Census Tracts:**
    * Convert the census tracts into a raster grid where each pixel represents the population density.
    
3. **Zonal Sum for Taxi Zones:**
    * Sum the rasterized population densities within the boundary of each taxi zone.
    * Calculate the average population density for each taxi zone.

In [2]:
# Loading the shape file for tax zones
current_dir = os.getcwd()
shapefile_path = os.path.join(current_dir, '..', 'data', 'NYC Taxi Zones', 'geo_export_5d80cfbe-13fc-4d7a-a530-e88bb7a8d4ee.shp')
taxi_zones = gpd.read_file(shapefile_path)
taxi_zones = taxi_zones.to_crs(epsg=4326)
taxi_zones.head()

,borough,location_i,objectid,shape_area,shape_leng,zone,geometry
0,EWR,1.0,1.0,0.000782,0.116357,Newark Airport,"POLYGON ((-74.18445 40.69500, -74.18449 40.695..."
1,Queens,2.0,2.0,0.004866,0.433470,Jamaica Bay,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,Bronx,3.0,3.0,0.000314,0.084341,Allerton/Pelham Gardens,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,Manhattan,4.0,4.0,0.000112,0.043567,Alphabet City,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,Staten Island,5.0,5.0,0.000498,0.092146,Arden Heights,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."


In [3]:
# Importing the census tracts file
census_tracts_path = os.path.join(current_dir, '..', 'data', 'nyct2020_24b','nyct2020.shp')
census_tracts = gpd.read_file(census_tracts_path)
census_tracts = census_tracts.to_crs(epsg=4326)
census_tracts.head()

,CTLabel,BoroCode,BoroName,CT2020,BoroCT2020,CDEligibil,NTAName,NTA2020,CDTA2020,CDTANAME,GEOID,PUMA,Shape_Leng,Shape_Area,geometry
0,1,1,Manhattan,000100,1000100,None,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,4121,10833.043929,1.843005e+06,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
1,2.01,1,Manhattan,000201,1000201,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000201,4103,4754.495247,9.723121e+05,"POLYGON ((-73.98450 40.70951, -73.98655 40.709..."
2,6,1,Manhattan,000600,1000600,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000600,4103,6976.286215,2.582705e+06,"POLYGON ((-73.99022 40.71440, -73.98934 40.714..."
3,14.01,1,Manhattan,001401,1001401,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,4103,5075.332000,1.006117e+06,"POLYGON ((-73.98837 40.71645, -73.98754 40.716..."
4,14.02,1,Manhattan,001402,1001402,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001402,4103,4459.156019,1.226206e+06,"POLYGON ((-73.98507 40.71908, -73.98423 40.718..."


In [8]:
# Importing the merged shape file with population density data
merged_shapefile_path = os.path.join(current_dir, '..', 'data', 'nyc_censust_pop_density_out', 'nyc_census_tracts_with_population.shp')
merged_shapefile = gpd.read_file(merged_shapefile_path)
merged_shapefile = merged_shapefile.to_crs(epsg=4326)

# Calculate the population density and add it as a column
merged_shapefile['population_density'] = merged_shapefile['P1_001N'] / (merged_shapefile['Shape_Area'] / 1e6)  # Assuming Shape_Area is in m^2

merged_shapefile.columns


Index(['CTLabel_x', 'BoroCode_x', 'BoroName', 'CT2020', 'BoroCT2020',
       'CDEligibil', 'NTAName', 'NTA2020', 'CDTA2020', 'CDTANAME', 'GEOID',
       'PUMA', 'Shape_Leng', 'Shape_Area', 'Key', 'P1_001N', 'NAME', 'state',
       'county', 'tract', 'CTLabel_y', 'County_1', 'StateName', 'Borough',
       'BoroCode_y', 'CTLabelNum', 'geometry', 'population_density'],
      dtype='object')

Bounds: minx=-74.25559136315213, miny=40.49613398761199, maxx=-73.70000906321272, maxy=40.91553277650282
Raster dimensions: width=0, height=0


ValueError: Calculated width and height must be > 0